## Initial code to test audio

In [ ]:
import sounddevice as sd
print(sd.query_devices())  # Find virtual cable's index


In [ ]:
import sounddevice as sd
import numpy as np

# Define parameters
duration = 5  # seconds
sample_rate = 16000  # Hz

# Record audio
print("Recording...")
audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype=np.float32)
sd.wait()  # Wait until recording is finished
print("Recording finished.")

# Play the recorded audio
print("Playing back the recorded audio...")
sd.play(audio_data, samplerate=sample_rate)
sd.wait()  # Wait until playback is finished
print("Playback finished.")

In [ ]:
print("Playing back the recorded audio...")
sd.play(audio_data, samplerate=sample_rate)
sd.wait()  # Wait until playback is finished
print("Playback finished.")

## Import dependencies

In [1]:
import sounddevice as sd
import numpy as np
import queue
from collections import deque
from whispercpp import Whisper
from openai import OpenAI
import soundfile as sf
import os
from io import BytesIO
import keyboard
from groq import Groq
import time
import pyaudio
import wave
import tempfile
import pyperclip
import webrtcvad
import threading

In [5]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


## Dependencies

In [3]:
pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
    --------------------------------------- 6.8/376.0 MB 42.0 MB/s eta 0:00:09
   - -------------------------------------- 11.8/376.0 MB 29.5 MB/s eta 0:00:13
   - -------------------------------------- 14.2/376.0 MB 24.7 MB/s eta 0:00:15
   - -------------------------------------- 15.7/376.0 MB 20.6 MB/s eta 0:00:18
   - -------------------------------------- 16.5/376.0 MB 16.3 MB/s eta 0:00:23
   - -------------------------------------- 16.8/376.0 MB 14.9 MB/s eta 0:00:25
   - -------------------------------------- 17.3/376.0 MB 12.1 MB/s eta 0:00:30
   - -------------------------------------- 17.8/376.0 MB 11.0 MB/s eta 0:00:33
   - -------------------------------------- 18.4/376.0 MB 10.1 MB/s eta 0:00:36
   -- ------------------------------------- 19.1/376.0 MB 9.2 MB/s eta 0:00:39
   -- ------------------------------------- 19.7/376.0 MB 8.6 MB/s

In [ ]:
pip install webrtcvad-wheels

In [ ]:
pip install PyAudio

In [ ]:
pip install groq

In [ ]:
pip install keyboard

In [ ]:
pip install soundfile

In [ ]:
pip install sounddevice numpy whispercpp


In [ ]:
pip install transformers

In [ ]:
pip install openai

## Testing local transcription

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load processor and model
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")


In [ ]:
from whispercpp import Whisper
whisper_model = Whisper.from_pretrained("base.en")  # Note method change


In [ ]:
# Initialize Whisper model (tiny.en for low latency)
whisper_model = Whisper("tiny.en")  # Replace with "base.en" or larger models if needed

# Configuration
SAMPLERATE = 16000  # Whisper works best at this rate
BLOCKSIZE = 1024    # 64ms chunks (tweak for lower latency)
DEVICE_ID = 1      # Replace with your virtual cable device index (use sd.query_devices())
audio_queue = queue.Queue(maxsize=10)  # Thread-safe buffer for audio chunks

# Callback function to capture audio chunks
def audio_callback(indata, frames, time, status):
    """Non-blocking audio chunk handler."""
    if status:
        print(f"Audio callback error: {status}")
    audio_queue.put(indata[:, 0].copy())  # Convert to mono and add to queue

# Process audio chunks in real-time using Whisper
def process_audio():
    """Continuously process audio chunks from the queue."""
    print("Starting transcription...")
    while True:
        try:
            # Get an audio chunk from the queue
            chunk = audio_queue.get()
            # Transcribe the chunk using Whisper.cpp
            text = whisper_model.transcribe(chunk.astype(np.float32))
            if text.strip():  # If transcription is not empty
                print(f"Transcribed Text: {text}")
                # Example: Trigger response for specific keywords
                if "memoization" in text.lower():
                    print("[Response] Memoization is an optimization technique...")
        except Exception as e:
            print(f"Error during transcription: {e}")

# Start the real-time audio stream and transcription loop
def start_real_time_transcription():
    """Start the audio stream and transcription."""
    try:
        print("Initializing audio stream...")
        with sd.InputStream(samplerate=SAMPLERATE, blocksize=BLOCKSIZE,
                            device=DEVICE_ID, channels=1, callback=audio_callback):
            process_audio()  # Start processing audio chunks in real-time
    except Exception as e:
        print(f"Error initializing audio stream: {e}")

if __name__ == "__main__":
    print("Starting real-time meeting assistant...")
    start_real_time_transcription()


## Testing online API transcription

In [2]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [6]:
# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": "you are a helpful assistant."
#         },
#         {
#             "role": "user",
#             "content": "Explain the importance of fast language models",
#         }
#     ],
#     model="llama-3.3-70b-versatile",
# )

# print(chat_completion.choices[0].message.content)

In [3]:
# Audio recording parameters
CHUNK = 480
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
VAD_AGGRESSIVENESS = 3  # 1-3 (3=most aggressive noise filtering)
MIN_SPEECH_DURATION = 1.2  # Seconds of speech to trigger processing
PRE_SPEECH_BUFFER = 2.0  # Seconds to capture before speech starts
POST_SPEECH_BUFFER = 1.5  # Seconds to capture after speech ends
MAX_BUFFER_SECONDS = 5 # Maximum buffer size in seconds

vad = webrtcvad.Vad(VAD_AGGRESSIVENESS)  # Aggressive noise filtering
audio_queue = queue.Queue()
processing_lock = threading.Lock()
in_speech = False
speech_start = 0 

In [4]:
processing_lock = threading.Lock()
audio_buffer = np.array([], dtype=np.int16)  # Declare and initialize audio_buffer globally
in_speech = False
speech_start = 0
buffer_start_idx = 0

def record_audio():
    """VAD based audio recording with threading."""
    global audio_buffer, in_speech, speech_start, buffer_start_idx
    
    def callback(in_data, frame_count, time_info, status):
        global audio_buffer, in_speech, speech_start, buffer_start_idx
        
        pcm = np.frombuffer(in_data, dtype=np.int16)

        with processing_lock:
            is_speech = vad.is_speech(pcm.tobytes(), RATE, len(pcm))
            
            if is_speech:
                if not in_speech:
                    # Speech start: track buffer position
                    speech_start = time.time()
                    buffer_start_idx = max(0, len(audio_buffer) - RATE//2)  # 0.5s lookback
                    in_speech = True
                audio_buffer = np.concatenate([audio_buffer, pcm])
            else:
                if in_speech:
                    # Speech end: calculate captured duration
                    captured_seconds = (len(audio_buffer) - buffer_start_idx) / RATE
                    if captured_seconds >= MIN_SPEECH_DURATION:
                        # Capture pre/post speech context
                        pre_samples = min(buffer_start_idx, int(RATE * PRE_SPEECH_BUFFER))
                        post_samples = int(RATE * POST_SPEECH_BUFFER)
                        
                        segment = audio_buffer[
                            buffer_start_idx - pre_samples : 
                            len(audio_buffer) + post_samples
                        ]
                        audio_queue.put(segment.copy())
                        
                    in_speech = False

        print(f"VAD: {'🗣️' if is_speech else '🔇'} | Buffer: {len(audio_buffer)/RATE:.1f}s", end='\r')
        return (None, pyaudio.paContinue)

    vad = webrtcvad.Vad(VAD_AGGRESSIVENESS)
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                    input=True, frames_per_buffer=CHUNK,
                    stream_callback=callback, start=False)
    stream.start_stream()
    return stream

def save_audio(audio_numpy):
    """Save numpy array directly"""
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        sf.write(f, audio_numpy, RATE, format='WAV', subtype='PCM_16')
        return f.name

def transcribe_audio(audio_numpy):
    """Direct memory-based transcription"""
    # Convert numpy array to bytes buffer
    buffer = BytesIO()
    audio_numpy = audio_numpy.astype(np.float32) / 32768.0 # Normalize to float32
    sf.write(buffer, audio_numpy, RATE, format='WAV', 
             subtype='FLOAT')
    buffer.seek(0)

    try:
        response = client.audio.transcriptions.create(
            file=("audio.wav", buffer.read()),
            model="whisper-large-v3-turbo",  # Verified correct model name
            response_format="text",
            language="en",
            temperature=0.2,
        )
        return response.strip()
    except Exception as e:
        print(f"⚠️ Transcription failed: {str(e)}")
        return ""

def play_audio(filename="recorded_audio.wav"):
    import pyaudio
    import wave

    wf = wave.open(filename, 'rb')
    p = pyaudio.PyAudio()

    # Open stream for playback
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Read and play back data in chunks
    chunk = 1024
    data = wf.readframes(chunk)
    while data:
        stream.write(data)
        data = wf.readframes(chunk)

    stream.stop_stream()
    stream.close()
    p.terminate()
    print("Playback finished.")

def process_transcription(text):
    """Process the transcribed text and generate a response."""
    cleaned = text.lower().strip()[:200] # Limit input length
    if not cleaned:
        return "No speech detected"
    keywords = {
        "memoization": "Memoization is an optimization technique that stores results of expensive function calls.",
        "recursion": "Recursion is a method where a function calls itself to solve smaller instances of a problem.",
        "dynamic programming": "Dynamic programming is a method for solving complex problems by breaking them down into simpler subproblems.",
        "algorithm": "An algorithm is a step-by-step procedure for calculations.",
        "data structure": "A data structure is a particular way of organizing and storing data in a computer.",
        "machine learning": "Machine learning is a subset of AI that enables systems to learn from data.",
        "artificial intelligence": "Artificial intelligence is the simulation of human intelligence in machines.",
        "deep learning": "Deep learning is a subset of machine learning that uses neural networks with many layers.",
        "natural language processing": "Natural language processing is a field of AI that focuses on the interaction between computers and humans through natural language.",
        "computer vision": "Computer vision is a field of AI that enables computers to interpret and understand visual information from the world.",
        "reinforcement learning": "Reinforcement learning is a type of machine learning where an agent learns to make decisions by taking actions in an environment to maximize cumulative reward.",
        "supervised learning": "Supervised learning is a type of machine learning where the model is trained on labeled data.",
        "unsupervised learning": "Unsupervised learning is a type of machine learning where the model is trained on unlabeled data.",
        "transfer learning": "Transfer learning is a technique where a model developed for one task is reused as the starting point for a model on a second task.",
        "chini": "Chini to pagal hai."
    }
    
    for kw in sorted(keywords.keys(), key=len, reverse=True):
        if kw in cleaned:
            return keywords[kw]
    return f"Command not recognized: {cleaned[:30]}..."

def copy_to_clipboard(text):
    """Copy text to clipboard."""
    pyperclip.copy(text)
    print("Response copied to clipboard!")


# def main():
#     while True:
#         input("Press Enter to start recording...")
#         audio_frames = record_audio()
#         temp_file_path = save_audio(audio_frames)
#         transcription = transcribe_audio(temp_file_path)
#         print("Transcription:", transcription)
#         os.remove(temp_file_path)

# if __name__ == "__main__":
#     main()

## Question detector

In [14]:
from transformers import pipeline, logging
logging.set_verbosity_error()  # Suppress warnings

class QuestionDetector:
    def __init__(self):
        # Lightweight model for real-time use (90MB)
        self.classifier = pipeline(
            "text-classification", 
            model="shahrukhx01/question-vs-statement-classifier"
        )

        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
        self.model = "llama3-70b-8192"
        
    def detect_question(self, text):
        """Returns True if input text is a question"""
        if not text.strip():
            return False
            
        result = self.classifier(text[:512])  # Trim to model's max length
        return result[0]['label'] == 'LABEL_1' and result[0]['score'] > 0.85
    
    def generate_answer(self, question, context=""):
        """Generate an answer to the question using Groq API"""
        try:
            response = self.client.chat.completions.create(
                messages=[{"role": "system", "content": f"""You are an expert interview coach. Generate concise, professional responses using:
                        - STAR method (Situation, Task, Action, Result)
                        - Max 3 sentences
                        - Incorporate keywords: {context}
                        - Maintain a professional tone"""},
                          {"role": "user", "content": f"Suggest response to: {question}"}],
                model=self.model,
                temperature=0.2,
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error generating answer: {e}")
            return "Sorry, I couldn't generate an answer."

In [17]:
texts = [
    "What is your greatest strength?", 
    "Tell me about a time you solved a problem.",
    "I worked at XYZ Corp for 5 years.",
    "Why did you leave your previous job?",
    "It was great working with my team."
]
context = "AWS, Python, React, Team Leadership"

qd = QuestionDetector()
for text in texts:
    print(f"Text: {text}")
    if qd.detect_question(text):
        print("Detected as a question.")
        response = qd.generate_answer(text, context)
        print(f"Generated Answer: {response}\n")
    else:
        print("Detected as a statement.")


Text: What is your greatest strength?
Detected as a question.
Generated Answer: Here's a possible response:

"My greatest strength is my ability to lead cross-functional teams in delivering scalable and efficient solutions, leveraging my technical expertise in AWS and Python. In my previous role, I successfully led a team of engineers in developing a cloud-based application using React, resulting in a 30% increase in deployment speed and a 25% reduction in costs. This experience has taught me the importance of effective team leadership, and I'm confident in my ability to drive similar results in this role."

Text: Tell me about a time you solved a problem.
Detected as a statement.
Text: I worked at XYZ Corp for 5 years.
Detected as a statement.
Text: Why did you leave your previous job?
Detected as a question.
Generated Answer: Here's a possible response:

"I left my previous role as a Senior Software Engineer at XYZ Corporation due to a desire to expand my technical skills in cloud co

## Main Workflow

In [9]:
FULL_TRANSCRIPT = []  # Simple list to store all spoken phrases
LAST_RESPONSE = ""

def main():
    global FULL_TRANSCRIPT, LAST_RESPONSE

    # Initialize detector
    qd = QuestionDetector()
    
    stream = record_audio()
    print("🎤 Listening... (Press Ctrl+C to stop)")
    
    try:
        while True:
            try:
                audio_data = audio_queue.get(timeout=0.2)
                
                if audio_data.size >= int(RATE * 0.3):  # Minimal audio threshold
                    # Core pipeline
                    transcription = transcribe_audio(audio_data)
                    # response = process_transcription(transcription)

                    # if question detected
                    if qd.detect_question(transcription):
                        response = client.chat.completions.create(
                            messages=[
                                {
                                    "role": "user",
                                    "content": transcription,
                                }
                            ],
                            model="llama-3.3-70b-versatile",
                        ).choices[0].message.content
                        print(f"💬 Response: {response}")
                    
                    # Simple storage
                    FULL_TRANSCRIPT.append(transcription)
                    # LAST_RESPONSE = response
                    
                    # Immediate feedback
                    print(f"\n📝 Heard: {transcription}")
                    # print(f"💬 Response: {response}")
                    # pyperclip.copy(response)
                    
            except queue.Empty:
                continue
                
    except KeyboardInterrupt:
        print("\n🛑 Stopping...")
        
    finally:
        stream.stop_stream()
        stream.close()
        
        # Simple verification
        print("\n=== COMPLETE TRANSCRIPT ===")
        for i, phrase in enumerate(FULL_TRANSCRIPT, 1):
            print(f"{i}. {phrase}")

if __name__ == "__main__":
    main()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Device set to use 0


🎤 Listening... (Press Ctrl+C to stop)
VAD: 🔇 | Buffer: 1.9ss
📝 Heard: Hi, how are you?
VAD: 🔇 | Buffer: 4.2ss
📝 Heard: Hi, how are you? So what is up?
VAD: 🗣️ | Buffer: 8.2s
📝 Heard: so what is up? also from denoiser import enhance
VAD: 🔇 | Buffer: 10.2ss
📝 Heard: Also from denoiser import enhance another tool to add on top of it to remove noise.
VAD: 🔇 | Buffer: 14.2ss
📝 Heard: on top of it to remove noise. Basically you can remove noise using this tool
VAD: 🗣️ | Buffer: 15.3s
📝 Heard: remove noise using these toolkits and you know
VAD: 🔇 | Buffer: 16.5ss
🛑 Stopping...

=== COMPLETE TRANSCRIPT ===
1. Hi, how are you?
2. Hi, how are you? So what is up?
3. so what is up? also from denoiser import enhance
4. Also from denoiser import enhance another tool to add on top of it to remove noise.
5. on top of it to remove noise. Basically you can remove noise using this tool
6. remove noise using these toolkits and you know
